# Import Libraries

In [1]:
from swarmintelligence import ParticleSwarmOptimization, Utilization
import os
import cv2
import matplotlib.pyplot as plt
import pandas as pd
import time # calculate CPU time
import pickle
import itertools

# import package metrics
from sewar.full_ref import *

# Load Data Testing

In [2]:
# load the model from disk
test_dataset_misc = pickle.load(open('datasets/test_dataset.pkl', 'rb'))
gray_misc_dataset = test_dataset_misc['gray']

In [3]:
gray_misc_dataset

{'AirplaneF16.tiff': array([[ 65, 199, 193, ..., 166, 164, 148],
        [ 54, 196, 192, ..., 179, 155, 140],
        [ 54, 197, 194, ..., 190, 159, 141],
        ...,
        [ 67, 213, 213, ..., 184, 176, 151],
        [ 68, 214, 214, ..., 166, 184, 167],
        [ 89,  86,  88, ..., 112, 111, 107]], dtype=uint8),
 'Lena.png': array([[162, 162, 162, ..., 170, 155, 128],
        [162, 162, 162, ..., 170, 155, 128],
        [162, 162, 162, ..., 170, 155, 128],
        ...,
        [ 43,  43,  50, ..., 104, 100,  98],
        [ 44,  44,  55, ..., 104, 105, 108],
        [ 44,  44,  55, ..., 104, 105, 108]], dtype=uint8),
 'Male.tiff': array([[130, 138, 140, ..., 115, 113, 111],
        [139, 134, 131, ..., 110, 117, 116],
        [141, 127, 131, ..., 109, 108, 107],
        ...,
        [102, 103, 103, ...,  76,  80,  89],
        [106, 102, 109, ...,  72,  69,  82],
        [ 98, 105, 116, ...,  85,  81,  89]], dtype=uint8),
 'Mandrill.tiff': array([[145,  56,  49, ..., 113, 153, 177],

# Load Optimal Parameter

In [4]:
optimal_params = pickle.load(open('results/hyperparameter_tuning/PSO/optimal_params.pkl', 'rb'))
optimal_params

{'particleSize': 25,
 'maxIteration': 100,
 'phi1': 2.6,
 'phi2': 1.7,
 'inertia': 0.5}

# Evaluation

In [5]:
thresholds = [2,3,4,5]
fitness_functions = ['otsu']
objs = ['max']
n_runs = 30

In [6]:
pso_evaluation_results = {
    'image_name': [],
    'thresholds': [],
    'fitness_function': [],
    'obj':[],
    'Mean best_thresholds':[],
    'Mean CPU_time (seconds)': [],
    'Mean Fitness':[],
    'Mean MSE':[],
    'Mean RMSE':[],
    'Mean PSNR':[],
    'Mean SSIM':[],
    'Mean UQI':[],
    'Regions': [],
    'CPU_time (seconds)': [],
    'Fitness':[],
    'MSE':[],
    'RMSE':[],
    'PSNR':[],
    'SSIM':[],
    'UQI':[],
}
for image_name, image_array in gray_misc_dataset.items():
    for threshold in thresholds:
        for fitness_function in fitness_functions:
            for obj in objs:
                pso_evaluation_results['image_name'].append(image_name)
                pso_evaluation_results['thresholds'].append(threshold)
                pso_evaluation_results['fitness_function'].append(fitness_function)
                pso_evaluation_results['obj'].append(obj)
                print('\t','='*30)
                print("\t",image_name, "| Threshold = ", threshold)
                print('\t','='*30)
                
                # untuk setiap algorithm run sebanyak n_runs lalu hitung rata-rata hasilnya
                list_cpu_time = []
                fitness_values = []
                list_best_thresholds = []
                list_fitness = []
                list_mse = []
                list_rmse = []
                list_psnr = []
                list_ssim = []
                list_uqi = []
                for i_run in range(n_runs):
                    # PSO optimization
                    pso = ParticleSwarmOptimization(
                        k=threshold,
                        particleSize=optimal_params['particleSize'],
                        maxIteration=optimal_params['maxIteration'],
                        fitness_function=fitness_function,
                        phi1=optimal_params['phi1'],
                        phi2=optimal_params['phi2'],
                        inertia=optimal_params['inertia']
                    )
                    # Evaluation Metrics
                    st = time.time()
                    particles, best_thresholds = pso.fit_run(image_array)
                    et = time.time()
                    regions = Utilization().digitize(image_array, best_thresholds)
                    list_cpu_time.append(et-st)
                    list_best_thresholds.append(best_thresholds)
                    fitness_values.append(pso.get_params_training_()['best_fitness_tracking'][-1])
                    list_mse.append(mse(image_array, regions))
                    list_rmse.append(rmse(image_array, regions))
                    list_psnr.append(psnr(image_array, regions))
                    list_ssim.append(ssim(image_array, regions)[0])
                    list_uqi.append(uqi(image_array, regions))
                    print(pso.get_params_training_()['best_fitness_tracking'][-1])

                
                mean_best_thresholds = np.round(np.array(list_best_thresholds).mean(axis=0)).astype('int')
                mean_regions=Utilization().digitize(image_array, mean_best_thresholds)
                pso_evaluation_results['Mean best_thresholds'].append(mean_best_thresholds)
                pso_evaluation_results['Mean CPU_time (seconds)'].append(np.mean(np.array(list_cpu_time)))
                pso_evaluation_results['Mean Fitness'].append(np.mean(np.array(fitness_values)))
                pso_evaluation_results['Mean MSE'].append(np.mean(np.array(list_mse)))
                pso_evaluation_results['Mean RMSE'].append(np.mean(np.array(list_rmse)))
                pso_evaluation_results['Mean PSNR'].append(np.mean(np.array(list_psnr)))
                pso_evaluation_results['Mean SSIM'].append(np.mean(np.array(list_ssim)))
                pso_evaluation_results['Mean UQI'].append(np.mean(np.array(list_uqi)))
                pso_evaluation_results['Regions'].append(mean_regions)
                pso_evaluation_results['CPU_time (seconds)'].append(list_cpu_time)
                pso_evaluation_results['Fitness'].append(fitness_values)
                pso_evaluation_results['MSE'].append(list_mse)
                pso_evaluation_results['RMSE'].append(list_rmse)
                pso_evaluation_results['PSNR'].append(list_psnr)
                pso_evaluation_results['SSIM'].append(list_ssim)
                pso_evaluation_results['UQI'].append(list_uqi)
                print(f"Mean Fitness = {np.mean(np.array(fitness_values))}")

	 AirplaneF16.tiff | Threshold =  2
1948.7106129145502
1948.7183902321062
1948.7106129145502
1948.7183902321062
1948.7183902321062
1948.7183902321062
1948.7106129145502
1948.7183902321062
1948.6765950132303
1948.7183902321062
1948.7106129145502
1948.7106129145502
1948.7183902321062
1948.6196858202202
1948.7183902321062
1948.7183902321062
1948.7183902321062
1948.7183902321062
1948.7183902321062
1948.6409800416593
1948.7106129145502
1948.7183902321062
1948.7183902321062
1948.7183902321062
1948.7183902321062
1948.7183902321062
1948.7183902321062
1948.6717399702634
1948.7183902321062
1948.7183902321062
Mean Fitness = 1948.7080160991597
	 AirplaneF16.tiff | Threshold =  3
2024.2193999564352
2024.646060512922
2024.7811694541858
2024.7910711088512
2024.555865815046
2024.2953000685475
2024.5438362896882
2024.2950206709224
2024.805475833105
2024.3989659443648
2024.5545067579892
2024.7838782335534
2024.7401565921582
2024.4019069777582
2024.7401565921582
2024.805475833105
2024.7910711088512
2024.

3253.3462629207843
3245.6481756992116
3252.6347815583945
3254.4757949601276
Mean Fitness = 3252.2270896388623
	 Mandrill.tiff | Threshold =  2
1549.0271327708897
1549.0271327708897
1549.0271327708897
1549.0271327708897
1549.0271327708897
1549.0271327708897
1549.0271327708897
1549.0271327708897
1549.0271327708897
1549.0271327708897
1549.0271327708897
1548.9859586850848
1549.0271327708897
1549.0271327708897
1549.0271327708897
1549.0271327708897
1549.0271327708897
1549.0271327708897
1549.0271327708897
1549.0271327708897
1549.0271327708897
1549.0271327708897
1549.0271327708897
1549.0271327708897
1549.0271327708897
1549.0271327708897
1549.0271327708897
1548.9952130399997
1549.0271327708897
1549.0271327708897
Mean Fitness = 1549.0246963103334
	 Mandrill.tiff | Threshold =  3
1639.463360998692
1639.466171099567
1639.466171099567
1639.4542198585186
1638.8662421178692
1639.4006244533603
1639.466171099567
1639.1357358599644
1639.466171099567
1639.466171099567
1639.4542198585186
1639.466171099567

4217.969061608366
4217.958687837565
4217.961826601611
4217.466197895181
4217.397121257758
4217.863217966309
4217.980323755768
4217.856603559883
4217.983460413122
4217.965898094624
4217.682719734341
Mean Fitness = 4217.799693027005


## Visualize Results using DataFrame

In [7]:
pso_evaluation_results_df =pd.DataFrame(pso_evaluation_results)
pso_evaluation_results_df

,image_name,thresholds,fitness_function,obj,Mean best_thresholds,Mean CPU_time (seconds),Mean Fitness,Mean MSE,Mean RMSE,Mean PSNR,Mean SSIM,Mean UQI,Regions,CPU_time (seconds),Fitness,MSE,RMSE,PSNR,SSIM,UQI
0,AirplaneF16.tiff,2,otsu,max,"[113, 173]",0.672372,1948.708016,2267.378556,47.616962,14.575577,0.794513,0.940265,"[[113, 255, 255, 255, 255, 255, 255, 255, 255,...","[0.6887161731719971, 0.5445220470428467, 0.772...","[1948.7106129145502, 1948.7183902321062, 1948....","[2258.248394012451, 2270.6405181884766, 2258.2...","[47.52103107059496, 47.651238369936166, 47.521...","[14.593086508184399, 14.569319775617668, 14.59...","[0.7948012578894507, 0.7944112258870609, 0.794...","[0.9403934885504599, 0.9402201034707982, 0.940..."
1,AirplaneF16.tiff,3,otsu,max,"[94, 146, 192]",1.374413,2024.618117,1753.536313,41.870681,15.693550,0.775651,0.953603,"[[94, 255, 255, 192, 192, 192, 192, 192, 192, ...","[1.1966521739959717, 1.2687945365905762, 1.189...","[2024.2193999564352, 2024.646060512922, 2024.7...","[1634.7165031433105, 1718.3016662597656, 1746....","[40.43162751044423, 41.45240241843367, 41.7890...","[15.996379138385015, 15.779809496316854, 15.70...","[0.7569450360416561, 0.7677185922467746, 0.773...","[0.9556167421257433, 0.9539957601997507, 0.953..."
2,AirplaneF16.tiff,4,otsu,max,"[86, 131, 174, 204]",1.337460,2068.925184,1067.120887,32.633651,17.866167,0.733987,0.969493,"[[86, 204, 204, 204, 204, 204, 204, 204, 204, ...","[1.417405605316162, 1.4683928489685059, 1.4537...","[2069.8095124848383, 2068.370314777357, 2068.2...","[1037.7972412109375, 1143.0753479003906, 1276....","[32.21486056482222, 33.80939733122125, 35.7257...","[17.96967848997785, 17.5500550221468, 17.07117...","[0.733062987749361, 0.7370660658934128, 0.7384...","[0.9701314607973435, 0.968568517689577, 0.9657..."
3,AirplaneF16.tiff,5,otsu,max,"[73, 110, 146, 181, 205]",1.429161,2093.558400,903.834425,30.002540,18.605338,0.745698,0.975179,"[[73, 205, 205, 205, 181, 205, 205, 205, 205, ...","[1.4073379039764404, 1.4543983936309814, 1.495...","[2092.637344132006, 2093.2778455788325, 2094.0...","[874.7182197570801, 1134.378086090088, 745.020...","[29.575635576553214, 33.68052977745582, 27.295...","[18.712121882038005, 17.583225326464056, 19.40...","[0.7436527148538387, 0.7484898995424488, 0.751...","[0.9753675755786585, 0.9715279472344392, 0.978..."
4,Lena.png,2,otsu,max,"[93, 151]",1.367390,1961.753298,2798.952395,52.903221,13.661475,0.612792,0.883653,"[[255, 255, 255, 255, 255, 255, 255, 255, 255,...","[1.3688578605651855, 1.3709399700164795, 1.255...","[1961.7722159600003, 1961.8220125153634, 1961....","[2741.265396118164, 2813.567070007324, 2741.26...","[52.35709499311592, 53.04306806744237, 52.3570...","[13.751292769920404, 13.638230885124642, 13.75...","[0.6125900473142749, 0.6121817460412936, 0.612...","[0.884400965866291, 0.8833090574580458, 0.8844..."
5,Lena.png,3,otsu,max,"[81, 127, 171]",1.385857,2127.063944,1249.475139,35.342009,17.166429,0.706048,0.933165,"[[171, 171, 171, 171, 171, 171, 171, 171, 171,...","[1.4243521690368652, 1.4244980812072754, 1.413...","[2127.6330679455386, 2127.3966087789154, 2126....","[1221.3651390075684, 1298.5173301696777, 1327....","[34.94803483756373, 36.03494595763504, 36.4312...","[17.26234840944687, 16.996326107171154, 16.901...","[0.7078861216108434, 0.7062798552052552, 0.706...","[0.9342626895307022, 0.9322785380831662, 0.934..."
6,Lena.png,4,otsu,max,"[76, 114, 146, 181]",1.479265,2188.941347,805.499280,28.373256,19.075054,0.752419,0.952997,"[[181, 181, 181, 181, 181, 181, 181, 181, 181,...","[1.4533789157867432, 1.380368947982788, 1.4513...","[2190.8091102039616, 2189.3117523926567, 2189....","[850.0637855529785, 828.1812553405762, 853.644...","[29.155853366913796, 28.7781384967926, 29.2171...","[18.8362884612498, 18.949549642044232, 18.8180...","[0.7486438146139497, 0.7507307994641674, 0.747...","[0.952833400415237, 0.9493593555366734, 0.9510..."
7,Lena.png,5,otsu,max,"[73, 110, 136, 161, 190]",1.354341,2213.167515

# Save results

In [8]:
pickle.dump(pso_evaluation_results_df, open('results/evaluation/PSO/PSO_otsu_df.pkl', 'wb'))
pickle.dump(pso_evaluation_results, open('results/evaluation/PSO/PSO_otsu_dict.pkl', 'wb'))